In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt, freqz
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../..')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import get_binary_stimuli, get_sample_interval, load, save, to_string

## Load Neural Data

In [3]:
wakeup_dir = r'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolWakeup'
wakeup_sessions = os.listdir(wakeup_dir)

data_path = "../../__data__/VAR/Wakeup_all"
os.makedirs(data_path, exist_ok=True)


In [ ]:
iterator = tqdm(total = len(wakeup_sessions))
for index in range(len(wakeup_sessions)):
    savepath = os.path.join(data_path, f"results_{wakeup_sessions[index][:-4]}")
    if not os.path.exists(savepath):
        filename = os.path.join(wakeup_dir, wakeup_sessions[index])

        electrode_info, lfp, lfp_schema, session_info, trial_info, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'trialInfo', 'unitInfo'], verbose=False)
        dt = lfp_schema['smpInterval'][0]
        T = lfp.shape[0]
        window = 2.5
        stride = window

        PCA_dim = -1
        end_step = T - int(30/dt) # take off 30 seconds
    #     end_step = 5000

        results = {'params': {}}
        results['params']['PCA_dim'] = PCA_dim
        results['params']['window'] = window
        results['params']['stride'] = stride
        results['params']['filename'] = filename

        for area in np.unique(electrode_info['area']):
            areas = [area]
            unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]
            data = lfp[:end_step, unit_indices]
            results[area] = compute_eigs(data, dt, window, stride, return_sigma_norms=True, PCA_dim=PCA_dim)
            results[area]['end_step'] = end_step
            results[area]['data_sigma_norms'] = get_data_sigma_norms(data, dt, window, stride)
            del data

            save(results, os.path.join(data_path, f"results_{wakeup_sessions[index][:-4]}"))
    else:
        time.sleep(0.5)
    iterator.update()
iterator.close()

  0%|          | 0/21 [00:00<?, ?it/s]